In [1]:
filename = 'user-ct-test-collection-01.txt'

In [2]:
import urllib
import os
import gzip
import shutil

if os.path.isfile(filename):
    print 'Dataset found'
else:
    print 'Downloading dataset'
    urllib.urlretrieve('https://github.com/dimosr/Big_Data/raw/master/resources/user-ct-test-collection-01.txt.gz', filename + '.gz')
    print 'Download complete, unzipping'
    with gzip.open(filename + '.gz', 'rb') as f_in:
        with open(filename, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
    os.remove(filename + '.gz')        
    print 'Unzip complete'

Download complete, unzipping
Unzip complete


In [3]:
# REMOVE THIS AFTER TESTING
!head -1000 user-ct-test-collection-01.txt > user-ct-test-collection-01.txt.cpy
!mv user-ct-test-collection-01.txt.cpy user-ct-test-collection-01.txt

In [4]:
import pandas as pd

df = pd.read_csv(filename, sep='\t')
df.head()

,AnonID,Query,QueryTime,ItemRank,ClickURL
0,142,rentdirect.com,2006-03-01 07:17:12,NaN,NaN
1,142,www.prescriptionfortime.com,2006-03-12 12:31:06,NaN,NaN
2,142,staple.com,2006-03-17 21:19:29,NaN,NaN
3,142,staple.com,2006-03-17 21:19:45,NaN,NaN
4,142,www.newyorklawyersite.com,2006-03-18 08:02:58,NaN,NaN
